<a href="https://colab.research.google.com/github/eldiablo-data/tensorflow2learning/blob/master/notebooks/Fashion_MNIST_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
tf.__version__

'2.5.0'

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model

# Fashion MNIST
This dataset is more complex than the regular MNIST dataset. It contains 10 classes of clothing and apparel object images in gray scale of size 28 x 28. We load this dataset from Keras similar to MNIST.

In [ ]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train_scaled = x_train / 255
x_test_scaled = x_test /255
print("x_train.shape:",x_train.shape)

x_train.shape: (60000, 28, 28)


The input is a 28 x 28 grayscale image. For convolution, colored images are expected (height, width and colors as 3 dimensions) and  it requires an additional dimension in the input. Hence, the scalar color is represented as a 1D vector.

In [ ]:
x_train_scaled_cnn = np.expand_dims(x_train_scaled, -1)
x_test_scaled_cnn = np.expand_dims(x_test_scaled,-1)
print("x_train_scaled_cnn.shape:",x_train_scaled_cnn.shape)

x_train.shape: (60000, 28, 28, 1)


In [ ]:
k = len(set(y_train))
print("Number of classes: ", k)

Number of classes:  10


## Modelling

In [ ]:
i = Input(shape=x_train_scaled_cnn[0].shape)
x = Conv2D(filters=32,kernel_size=(3,3),strides=2, activation='relu')(i)

We add a 2D convolution layer of size 32 with kernel size 3x3. This is a fine layer looking into patterns and strokes. In the next layer, we will increase the filters to look for larger patterns like, say buttons and stitches. In the final layer, we will increase the filter to even larger patterns such as collars and sleeves. 
Strides make the model skip a certain number of neighbouring pixels to save computation times. Usually immediate neighbours tend to have similary color values. 